#### General imports and definitions

In [ ]:
import tensorflow as tf
import numpy as np
import os
import shutil
import random
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.utils import class_weight

import matplotlib
import matplotlib.pyplot as plt
plt.rc('font', size=16) 

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)
print(tf.config.list_physical_devices())

Suppress warnings

In [ ]:
import warnings
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

Set seed for reproducibility

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

#### Load and clean the dataset

In [ ]:
# Load the .npy file
X = np.load(r'../input/training/x_train.npy')
y = np.load(r'../input/training/y_train.npy').astype(np.int8)

# Note as sequences are already built, with window 36
print(X.shape, X.dtype, sep=", ")   
print(y.shape, y.dtype, sep=", ") 

print(X[0][0])

#### Drop features

In [ ]:
def keep_only(X, features_list):
    n_instances, n_timestamps, n_features = X.shape
    X = np.reshape(X, newshape=(-1, n_features))
    temp = np.empty((n_instances * n_timestamps, len(features_list)))
    
    i = 0
    for f in features_list:
        temp[:, i] = X[:, f]
        i += 1
    
    temp = np.reshape(temp, newshape=(n_instances, n_timestamps, len(features_list)))
    return temp

drop = False

if drop:
    X = keep_only(X, [1, 2, 3, 4, 5])
    
print(X.shape)
print(X[0][0])

In [ ]:
# Compute the class weights in order to balance loss during training
labels = np.unique(np.fromiter([t for t in y], np.int32))
class_weights = dict(enumerate(class_weight.compute_class_weight('balanced', classes=labels, y=y)))
print(class_weights, '\n')

# Convert the sparse labels to categorical values
y = tfk.utils.to_categorical(y)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

#### Metadata

In [ ]:
input_shape = X_train.shape[1:]
n_timestamps, n_features = input_shape
classes = y_train.shape[-1]
label_mapping = {
    'Wish': 0,
    'Another': 1,
    'Comfortably': 2,
    'Money': 3,
    'Breathe': 4,
    'Time': 5,
    'Brain': 6,
    'Echoes': 7,
    'Wearing': 8,
    'Sorrow': 9,
    'Hey': 10,
    'Shine': 11,  
}

# For training
batch_size = 512
epochs = 1000

# Define the scaler we intend to use
scaler = StandardScaler()

# Define the K-fold Cross Validator
num_folds = 10
kfold = KFold(n_splits=num_folds, shuffle=True)

#### Data scaling

In [ ]:
def scale(X_train, X_test, scaler):
    n_instances_train = X_train.shape[0]
    X_train = np.reshape(X_train, newshape=(-1, n_features))
    X_train = scaler.fit_transform(X_train)
    X_train = np.reshape(X_train, newshape=(n_instances_train, n_timestamps, n_features))
    
    if X_test is not None:
        n_instances_test = X_test.shape[0]
        X_test = np.reshape(X_test, newshape=(-1, n_features))
        X_test = scaler.transform(X_test)
        X_test = np.reshape(X_test, newshape=(n_instances_test, n_timestamps, n_features))

scaling = True

if scaling:
    scale(X_train, X_test, scaler)

#### Support functions

In [ ]:
def plot_history(history):
    best_epoch = np.argmax(history['val_accuracy'])
    plt.figure(figsize=(17,4))
    plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
    plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
    plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
    plt.title('Categorical Crossentropy')
    plt.legend()
    plt.grid(alpha=.3)
    plt.show()

    plt.figure(figsize=(17,4))
    plt.plot(history['accuracy'], label='Training accuracy', alpha=.8, color='#ff7f0e')
    plt.plot(history['val_accuracy'], label='Validation accuracy', alpha=.9, color='#5a9aa5')
    plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
    plt.title('Accuracy')
    plt.legend()
    plt.grid(alpha=.3)
    plt.show()

    plt.figure(figsize=(17,4))
    plt.plot(history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
    plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
    plt.legend()
    plt.grid(alpha=.3)
    plt.show()

def plot_cm(model):
    # Predict the test set with the LSTM
    predictions = model.predict(X_test)
    predictions.shape

    # Compute the confusion matrix
    cm = confusion_matrix(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1))

    # Compute the classification metrics
    accuracy = accuracy_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1))
    precision = precision_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')
    recall = recall_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')
    f1 = f1_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average=None)
    print('Accuracy:',accuracy.round(4))
    print('Precision:',precision.round(4))
    print('Recall:',recall.round(4))
    print('F1:',f1.round(4))

    # Plot the confusion matrix
    plt.figure(figsize=(10,8))
    sns.heatmap(cm.T, cmap='Blues', xticklabels=list(label_mapping.keys()), yticklabels=list(label_mapping.keys()), annot=True, fmt="d")
    plt.xlabel('True labels')
    plt.ylabel('Predicted labels')
    plt.show()

#### Classifier

In [ ]:
# we advice to choose what parameters to change and include them directly among the attributes of this function
def build_TS_classifier(input_shape, classes):
    
    input_layer = tfkl.Input(shape=input_shape)
    
    x = tfkl.Conv1D(
        filters = 256,
        kernel_size = 3,
        padding = 'same',
        activation = 'relu'
        )(input_layer)
    
    x = tfkl.Conv1D(
        filters = 256,
        kernel_size = 3,
        padding = 'same',
        activation = 'relu'
        )(x)
    
    #x = tfkl.MaxPooling1D()(x)

    x = tfkl.Conv1D(
        filters = 512,
        kernel_size = 3,
        padding = 'same',
        activation = 'relu'
        )(x)
    
    #x = tfkl.MaxPooling1D()(x)

    x = tfkl.Conv1D(
        filters = 256,
        kernel_size = 3,
        padding = 'same',
        activation = 'relu'
        )(x)
    
    x = tfkl.Conv1D(
        filters = 256,
        kernel_size = 3,
        padding = 'same',
        activation = 'relu'
        )(x)
    
    x = tfkl.GlobalAveragePooling1D()(x)
    
    # Classifier
    x = tfkl.Dense(
        units = 256,  
        activation = 'relu'
        )(x)
    
    x = tfkl.Dropout(0.2, seed=seed)(x)

    x = tfkl.Dense(
        units = 128,  
        activation = 'relu'
        )(x)
    
    x = tfkl.Dropout(0.2, seed=seed)(x)
    
    x = tfkl.Dense(
        units = 128,  
        activation = 'relu'
        )(x)
    
    x = tfkl.Dropout(0.2, seed=seed)(x)

    output_layer = tfkl.Dense(classes, activation='softmax')(x)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='cnn_model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-3), metrics='accuracy')

    # Return the model
    return model

In [ ]:
cross_val = True
final_train = True

if cross_val:
    best_acc = 0
    best_model = None
    best_history =  []
    
    # grid search FORs go here (remember to indent the following code: SEE THEN)
    #print(f'°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°° MODEL {i} °°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°')
    
    print(f'°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°° START °°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°')

    acc_per_fold = []
    fold_no = 1
    for train, test in kfold.split(X, y):
        print(f'Training for fold {fold_no} ...', end=' ')

        # Intantiate a new model
        model = build_TS_classifier(input_shape, classes)
        
        # Apply scaling
        X_train = np.copy(X[train])
        y_train = np.copy(y[train])
        X_test = np.copy(X[test])
        y_test = np.copy(y[test])
        scale(X_train, X_test, scaler)

        history = model.fit(X_train, y_train,
                  batch_size = batch_size,
                  epochs = epochs,
                  validation_data = (X_test, y_test),
                  #class_weight = class_weights,
                  verbose = 0,
                  callbacks = [
                      tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=51, restore_best_weights=True),
                      tfk.callbacks.ReduceLROnPlateau(monitor='val_accuracy', mode='max', patience=5, factor=tf.math.exp(-0.1), min_lr=2e-5)
                  ]
        ).history

        # Generate generalization metrics
        acc = np.max(history['val_accuracy']) * 100
        print(f'score: accuracy of {acc:.3f}%')
        acc_per_fold.append(acc)

        # Increase fold number
        fold_no = fold_no + 1
        if fold_no <= num_folds + 1:
            print('------------------------------------------------------------------------')

    print('\nAverage scores for all folds:')
    acc = np.mean(acc_per_fold)
    print(f'> Accuracy: {acc} (+- {np.std(acc_per_fold)})')
    if acc > best_acc:
        best_acc = acc
        best_model = model  # the last model is taken
        best_history = history
    
    # INDENT TILL THE PREVIOUS LINE
    print('°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°')
              
    # Plot results
    best_model.summary()
    plot_history(best_history)

    # RETRAIN THE BEST MODEL ON THE WHOLE DATASET
    if final_train:
        
        # Reinstanciate the model (if we have parameters we must pass them over)
        best_model = build_TS_classifier(input_shape, classes)
        
        # Apply scaling
        scale(X, None, scaler)
        
        best_model.fit(
            x = X,
            y = y,
            batch_size = batch_size,
            epochs = epochs,
            #class_weight = class_weights,
            callbacks = [
                tfk.callbacks.EarlyStopping(monitor='accuracy', mode='max', patience=81, restore_best_weights=True),
                tfk.callbacks.ReduceLROnPlateau(monitor='val_accuracy', mode='max', patience=10, factor=tf.math.exp(-0.1), min_lr=2e-5)
            ]
        )
    
else:
    best_model = build_TS_classifier(input_shape, classes)
    best_model.summary()
    
    # Train the model
    best_history = best_model.fit(
        x = X_train,
        y = y_train,
        batch_size = batch_size,
        epochs = epochs,
        validation_data = (X_test, y_test),
        #class_weight = class_weights,
        callbacks = [
            tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=81, restore_best_weights=True),
            tfk.callbacks.ReduceLROnPlateau(monitor='val_accuracy', mode='max', patience=10, factor=tf.math.exp(-0.1), min_lr=2e-5)
        ]
    ).history

    # Plot results
    plot_history(best_history)
    plot_cm(best_model)

In [ ]:
# Needed to write correct preprocessing code (meaningful only if the "final_train" flag is set,
# otherwise simply returns the scaler fit on the last fold)
print('[', end = '')
print(*scaler.mean_, sep=',', end = '')
print(']')
print('[', end = '')
print(*scaler.scale_, sep=',', end = '')
print(']')
print('[', end = '')
print(*scaler.var_, sep=',', end = '')
print(']')

In [ ]:
save = True

if save:
    best_model.save('best_model', include_optimizer=False)
    shutil.make_archive('best_model', 'zip', 'best_model')